In [32]:
import pandas as pd

In [33]:
boroughs = ['Camden', 'Hackney', 'Hammersmith and Fulham', 'Haringey', 'Islington', 'Kensington and Chelsea', 'Lambeth', 'Lewisham', 'Newham', 'Southwark', 'Tower Hamlets', 'Wandsworth', 'Westminster', 'Barking and Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley', 'Croydon', 'Ealing', 'Enfield', 'Greenwich', 'Harrow', 'Havering', 'Hillingdon', 'Hounslow', 'Kingston upon Thames', 'Merton', 'Redbridge', 'Richmond upon Thames', 'Sutton', 'Waltham Forest']

In [34]:
# period_start = "01-10-2018"
# period_end = "30-09-2019"
def process_rental_data(filepath, period_start, period_end):
    # print(f'{filepath=}')
    with pd.ExcelFile(filepath) as input_file:
        # print(f'{input_file=}')
        sheets = [s for s in input_file.sheet_names if s.startswith(('Table2', 'Table 2'))]
        for sheet in sheets:
            property_type = pd.read_excel(input_file, sheet_name=sheet, header=None, usecols="E", skiprows=lambda x: x != 5).iat[0, 0]

            # print(f'{property_type=}')
            # exit()

            data = pd.read_excel(input_file, sheet_name=sheet, usecols="D,F,H", na_values=['.', '..'], skiprows=lambda x: x not in range(6, 364))

            is_london_borough = [x in boroughs for x in data['Area']]
            # print(is_london_borough)
            filtered_by_borough = data[is_london_borough].copy()

            filtered_by_borough['Period Start Date'] = period_start
            filtered_by_borough['Period End Date'] = period_end
            filtered_by_borough['Property Type'] = property_type

            # display(filtered_by_borough)

            output_file = "./converted_data/rent_data.csv"
            if not isfile(output_file):
                filtered_by_borough.to_csv(output_file, index=False)
            else:
                filtered_by_borough.to_csv(output_file, index=False, mode='a', header=False)

In [35]:
from os import listdir
from os.path import isfile, join

In [36]:
import re
def split_date(date_string):
    return date_string[4:] + '/' + date_string[2:4] + '/' + date_string[:2]

def parse_rental_data_dates(filename):
    date_range = re.search("(\d{8})-(\d{8})", filename)
    start_date = split_date(date_range.group(1))
    end_date = split_date(date_range.group(2))

    return [start_date, end_date]

In [37]:
rental_data_path = "./raw-data/rental-data/"
rental_file_paths = [file for file in listdir(rental_data_path) if isfile(join(rental_data_path, file))]

for filename in rental_file_paths:
    start_date, end_date = parse_rental_data_dates(filename)
    process_rental_data(join(rental_data_path, filename), start_date, end_date)